## Requirements

In [ ]:
# Install the necessary libraries!pip install -q "openai>=1.0.0" \               pypdf \               langchain-text-splitters \               sentence-transformers \               faiss-cpu

## Open Router Chat Implementation

In [ ]:
from openai import OpenAIclient = OpenAI(  base_url="https://openrouter.ai/api/v1",  api_key="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",)completion = client.chat.completions.create(  extra_body={},  model="qwen/qwen3-235b-a22b:free",  messages=[    {      "role": "user",      "content": "What is the meaning of life?"    }  ])print(completion.choices[0].message.content)

## Open Router RAG Implementation

In [ ]:
import osfrom google.colab import userdatafrom openai import OpenAI# Load your secure API key from Colab SecretsOPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')OPENROUTER_BASE_URL = userdata.get('OPENROUTER_BASE_URL')# Initialize the OpenAI client to connect to OpenRouterclient = OpenAI(  base_url=OPENROUTER_BASE_URL,  api_key=OPENROUTER_API_KEY,)

In [ ]:
from langchain_community.document_loaders import PyPDFLoaderfrom langchain_text_splitters import RecursiveCharacterTextSplitterfrom langchain_community.embeddings import HuggingFaceEmbeddingsfrom langchain_community.vectorstores import FAISS# 1. Load your document (make sure to upload "my_document.pdf")loader = PyPDFLoader("my_document.pdf")documents = loader.load()print(f"Loaded {len(documents)} pages from the document.")# 2. Split the document into manageable chunkstext_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)docs = text_splitter.split_documents(documents)print(f"Document split into {len(docs)} chunks.")# 3. Create free embeddingsmodel_name = "sentence-transformers/all-MiniLM-L6-v2"embeddings = HuggingFaceEmbeddings(model_name=model_name)# 4. Create a FAISS vector store to enable searchingvectorstore = FAISS.from_documents(docs, embeddings)retriever = vectorstore.as_retriever()print("Vector store and retriever are ready.")

In [ ]:
def answer_query(query: str):    """    Performs the RAG process: Retrieve, Augment, and Generate.    """    # 1. Retrieve: Get the most relevant documents from the vector store    retrieved_docs = retriever.invoke(query)    # 2. Augment: Format the retrieved documents into a context string    context = "\n\n".join([doc.page_content for doc in retrieved_docs])    # 3. Augment: Create a detailed prompt for the LLM    prompt_template = f"""    You are an expert assistant. Use the following context to answer the question accurately.    If the answer is not found in the context, state that you don't know.    CONTEXT:    {context}    QUESTION:    {query}    ANSWER:    """    print("--- Sending prompt to LLM ---")    # 4. Generate: Send the prompt to the OpenRouter model    completion = client.chat.completions.create(      model="mistralai/mistral-7b-instruct:free", # Using a reliable free model      messages=[        {          "role": "user",          "content": prompt_template,        }      ]    )    return completion.choices[0].message.content

In [ ]:
# Ask a question about the content of your PDFmy_question = "What is the main conclusion of this document?"# Get the answer from your RAG applicationfinal_answer = answer_query(my_question)print("\n--- Final Answer ---")print(final_answer)